# Metody kernelowe

<h3 id=tocheading>Spis treści</h1>
<div id="toc"></div>

In [4]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [23]:
# -*- coding: utf-8 -*-

from __future__ import unicode_literals
from __future__ import print_function
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.io import gridplot, output_file, show, hplot, vplot
from bokeh.plotting import figure, output_notebook, vplot
from bokeh.charts import Scatter

In [24]:
output_notebook()

Loading BokehJS ...

# Uogólnione modele liniowe
---
* uogólniony model liniowy ma zwykle postać
$$\hat{y}(x)=\theta_0+\sum_{k=1}^K\theta_k\phi_k(x)$$
gdzie $\phi_k()$ są __wstępnie zdefiniowanymi__ nieliniowymi funkcjami
  * $\phi_k()$ mogą być wielomianami
  * jeśli funkcje będą miały postać $x_1^{p_1}x_2^{p_2}\dots x_l^{p_l}$ z ograniczeniem $p_1+p_2+\dots+p_m\leq r$, to razem czynników bedzie $$K=\frac{(l+r)!}{r!}$$
    * dla $l=10, r=3$ jest to już $K=286$
      * będzie więc bardzo dużo parametrów do ustalenia
  * użycie wielomainów jest uzasadnione tym, że każda ciągła funkcja może być dowolnie dokładnie aproksymowana wielomianami
  
* takie funkcje bazowe muszą być jednak __wstępnie__ wybrane
  * funkcje są także sutalone dla danych i __nie są__ zależne od ich postaci
  * wykorzystanie __ustalonych__ funkcji uniemożliwia zminimalizowanie błędu __poniżej__ pewnego progu
  * rozwiązaniem są __funkcje zależne od oryginalnych danych__
    * przykładem rozwiązania są sieci neuronowe
    
  * w takich przypadkach
    * modele stają się nieliniowe
    * są jednak trudniejsze w optymalizaji

# Twierdzenie Covera
* rozwinięcie wejscia o ustalone funkcje bazowe
  * pozwala osiągnąć lepszy błąd w problemach regresji
  * argumenty teorii aproksymacji nie mają zastosowania w klasyfikacji
    * niech dany będzie problem binarny $y\in\{-1,+1\}$
    * __nie jest__ istotne by $\hat{y}$ było (prawie) równe $+1$ czy $-1$
    * wystarczy, że jest bliższe poprawnej odpowiedzi, niż niepoprawnej
    
    
    ---
* punkty $x_1,x_2,\dots,x_N\in\mathbb{N}^l$ są w __uogolnionym położeniu__ (ang. general position), jeśli __nie__ istnieje podzbiór $l+1$ punktów leżących na $(l-1)$-wymiarowej hiperpłaszczyźnie
  * jeśli punkty są na płaszczyźnie, to __nie__ mogą 3 punkty leżeć na jednej prostej
  * jeśli w $\mathbb{R}^3$, to 4 punkty __nie__ mogą leżeć na jednej hiperpłaszczyźnie
  > Twierdzenie (Covera) Liczba wszystkich możliwychgrupowań (__dychotomii__) $O(N, l)$, które mogą być utworzone przez $(l-1)$-wymiarowe hiperpłaszczyzny dla rozdzielenia $N$ punktów na __dwie__ klasy jest dana przez $$O(N,l)=2\sum_{i=0}^l\binom{N-l}{i}$$ 
  gdzie 
  
  $$\binom{N-l}{i}=\frac{(N-1)!}{(N-i-1)!i!}$$
  
    * to jest podział __liniowy__
      * stąd mówimy o problemach __liniowo separowalnych__
    * każdy podział jest liczony dwukrotnie: punktu z jednej klasy mogą należec do klasy $c_1$ lub do $c_2$
    * dla 4 punktów na płaszczyźnie mamy $O(4,2)=14$
    * wszystkich możliwych kombinacji $N$ punktów na 2 grupy jest $2^N=16$
      * a więc niektórych brakuje?
    * dla $N\leq l+1$ zachodzi $O(N,l)=2^N$ (a więc wszystkie)
    
  > Dla $N$ punktów w $l$-wymiarowej przestrzeni, prawdopodobieństwo podzielenia punktów na dwie liniowo separowalne klasy wynosi 
  $$P(N,l)=\frac{O(N,l)}{2^N}=\left\{\array{\frac{1}{2^{N-1}}\sum_{i=0}^l\binom{N-1}{i}&N>l+1\\1&N\leq l+1}\right.$$

  <img src="cover_theorem.png" width="90%"/> [Za Theodoridis]
  
  * $r=N/(l+1)$ czyli stosunek liczby punktów do wymiarowości plus jeden
  * dla $r\leq1$ jest zawsze pewność podziału
  * dla $r=2$ czyli $N=2(l+1)$ prawdopodobieństwo jest zawsze $\displaystyle\frac{1}{2}$
    * $O(2(l+1),l)=2^{2l+1}$
  * dla przestrzeni o __wyższym wymiarze__ krzywa jest bardziej stroma
    * prawdopodobieństwo liniowego podziału dąży równomiernie do jedności wraz ze wzrostem wymiaru przestrzeni!
    
    
    
* jak można wykorzystać twierdzenie Covera?
  * utworzyć mapowanie
  $$\phi:\mathbb{R}^l\longrightarrow\mathbb{R}^K\;\;K\gg l$$
  * im większe $K$, tym większa powinna być szansa na rozdzielenie zadanych punktów
  * czy jednak na pewno jest to rozwiązanie na wszelkie problemy?
    * przykładowe 
    $$\phi:\mathbb{R}^2\ni[x_1,x_2]\longrightarrow
    [x_1,x_2,4\exp(-(x_1^2+x_2^2)/3)+5]\in\mathbb{R}^3$$
    <img src="cover_mapping.png" width="100%"/>[Za Theodoridis]
      * punkty są wyraźnie separowalne
      * punkty leżą na rozmaitości o __oryginalnym wymiarze__!
        * nie możemy oszukać natury "dodając" informację, której przecież nie ma!
        * możemy jednak "dodając" cechy wyekstrahować informację normalnie niedostępną dla klasyfikatorów danego typu
        

* w wysokich wymiarach trzeba dopasować dużą liczbę parametrów
  * to grozi nadmiernym dopasowaniem i słabą generalizacją
  * potrzebne są procedury ostrożnego zwiększania wymiaru
    * nowe wymiary muszą zależeć od danych
    * koniecznie trzeba zastosować silne procedury __regularyzacji__ dla redukcji zbędnych parametrów

# Postać dualna problemu
* metody zwykle były __parametryczne__: dla modelu wyszukiwany (uczony w iteracyjnym zwykle procesie) wektor __parametrów__ (albo __wag__ w sieciach neuronowych) i rozwiązanie ma postać $$\hat{y}=f(x, w)$$
* modele mogą także __zapamiętywać__ przykłady uczące (albo ich część) i budować z tego modele oparte na gęstości
  * Parzen z kombinacją funkcji ___jądra___ wokół przykładów uczących
  * k-sąsiadów
    * modele _pamięciowe_ z ustaloną funkcją odległości
    
* modele mogą być przedstawione w __równoważnej__ postaci __dualnej__
  * predykcje są liniowymi kombinacjami funkcji __jądra__ obliczanych w punktach danych $$k(x_1, x_2)=\Phi(x_1)^T\Phi(x_2)$$
* tzw. __kernel trick__ pozwala na budowę nowych funkcji jądra __bez__ konieczności budowy funkcji cech
  * funkcje jądra mogą zapewniać pewne żądane cechy, np. inwariantność na obrót, translację, itp.
  * ułatwienie obliczeń

## Perceptron
* Rosenblatt (1956) zaproponował model __perceptronu__
  * model uczony w trybie __online__
    * poprawianie błędów dla kolejnych przykładów
    * dla problemów __liniowo separowalnych__ algorytm __gwarantuje__ znalezienie rozwiązania
  * rozwiązuje problemy binarne dla $y\in\{-1,+1\}$
  $$y(x)=\left\{\array{-1&<w\cdot x>+b<0\\1&<w\cdot x>+b>0}\right.$$
  
  
  
-----
### postać pierwotna algorytmu uczenia perceptronu

* $y * (w^T x + b) >0$ oznacza __prawidłową__ klasyfikację
```python
w=0, b=0
R=max_i ||x_i||
modified = True
while modified:
    modified = False
    for i in range(N):
        if y_i*(w^T x + b) <= 0:
            modified = True
            w = w + eta y_i x_i
            b = b + eta y_i R^2
```      
  * co właściwie robi algorytm?
    * dla każdego __źle__ zaklasyfikowanego przykładu 
      * __dodaje__ $x_i$ ze stałą uczącą, jeśli przykład z klasy $+1$
      * __odejmuje__ $x_i$ ze stałą uczącą, jeśli przykład z klasy $-1$
    * odpowiada to __obrotowi__ hiperpłaszczyzny rozdzielającej
    * model ma więc postać $$\boxed{w=\sum_{i=1}^N\alpha_iy_ix_i\;}\tag{*}$$
  * znaleziona hiperpłaszczyzna $(w, b)$ jest płaszczyzną decyzyjną
  * najmniejszą odległość od jednego z przykładów od hiperpłaszczyzny nazywamy __marginesem funkcjonalnym__
    * model SVM będzie __maksymalizował__ margines funkcjonalny
    
### postać dualna algorytmu uczenia perceptronu
```python
alpha=0, b=0
R=max_i ||x_i||
modified = True
while modified:
    modified = False
    for i in range(N):
        if y_i * sum_k(alpha_i * y_i <x_k, x_i> + b) <= 0:
            modified = True
            alpha_i = alpha_i + 1
            b = b + y_i R^2
```      
  * jeśli przykład $x_i$ jest źle klasyfikowany
    * przykłady częściej źle klasyfikowane będą dodawane częściej
    * przykłady nie sprawiające kłopotów nie będą w ogóle dodawane
      * to oczywiscie zależy od procesu uczenia i kolejności
    * o rozwiązaniu będą stanowić przykłady __blisko__ granicy decyzyjnej
      * wartość $\alpha_i$ może służyć do rangowania przykładów ze względu na ich zawartość informacyjną
    * prawdopodobnie wystarczą przykłady __najbliższe__!
  * model ma postać
  $$\boxed{\begin{align}
  \hat{y}(x)&=sgn(w^Tx+b)\\
  &=sgn\left(<\sum_{j=1}^N\alpha_jy_jx_j\cdot x>+b\right)\\
  &=sgn\left(\sum_{j=1}^N\alpha_jy_j<x_j\cdot x>+b\right)
  \end{align}\;}$$
  
* dla większości modeli liniowych istnieje postać dualna
* dane pojawiają się wyłącznie jako elementy __macierzy Grama__
$$K=\Phi\Phi^T\;\; K_ij(x_i, x_j)=\phi(x_i)\phi(x_j)$$
* w postaci modelu dane uczące pojawiają się __jedynie__ przez istotność $\alpha_i$ ich iloczynu skalarnego z przykładem rozpoznawanym

## Postać rozwiązania dla modelu liniowej regresji

* model liniowy ma funkcję __energii__
$$\mathbb{E}(\theta)=\frac{1}{2}\sum_k(\theta^T\phi(x)_k-y_k)^2+\frac{\lambda}{2}\theta^T\theta$$
* stąd rozwiązanie dla __minimalizacji__ $\mathbb{E}(\theta)$
$$\begin{align}
\frac{\nabla\mathbb{E}}{\nabla \theta}&=\sum_k(\theta^T\phi(x)_k-y_k)\phi(x_k)+\lambda{}\theta\\
\theta&=-\frac{1}{\lambda}\sum_k(\theta^T\phi(x)_k-y_k)\phi(x_k)\\
&\hskip{30ex}\text{niech}\;u_k=-\frac{1}{\lambda}(\theta^T\phi(x)_k-y_k)\tag{**}\\
&=\sum_ku_k\phi(x_k)\\
\theta&=\Phi^Tu
\end{align}$$
* stąd nowa postać funkcji kosztu $\mathbb{E}(\theta)=\mathbb{E}(u)$
$$\begin{align}
\mathbb{E}(u)&=\frac{1}{2}\sum_k(u)\Phi^Tu\phi(x_n)-y_n)^2+\frac{\lambda}{2}u^T\Phi\Phi^Tu\\
&=\frac{1}{2}u^T\Phi\Phi^T\Phi\Phi^Tu-u^T\Phi\Phi^TY+\frac{1}{2}YY^T+\frac{\lambda}{2}u^T\Phi\Phi^Tu\\
&\hskip{11ex}\text{macierz}\; K=\Phi\Phi^T\; \text{jest symetryczną macierzą Grama}\\
&=\frac{1}{2}u^TKKu-u^TKY+\frac{1}{2}YY^T+\frac{\lambda}{2}u^TKu
\end{align}$$
* z (**) i $\theta=\Phi^Tu$
$$\begin{align}u=-\frac{1}{\lambda}(\theta^T\Phi-Y)&=-\frac{1}{\lambda}(u^T\Phi\Phi^T-Y)\\
&=-\frac{1}{\lambda}(u^TK-Y)\\
-\lambda u&=u^TK-Y\\
Y=u(K+\lambda{}I)\\
u=(K+\lambda{}I)^{-1}Y
\end{align}$$
co odpowiada rozwiązaniu w postaci __dualnej__
$$\boxed{
\begin{align}
\hat{y}(x)&=\theta^T\phi(x)\\
&=u^T\Phi\phi(x)=k(x)(K+\lambda{}I)^{-1}
\end{align}\;}$$
gdzie $k(x)=k(x,x_k)$ jest wektorem

* rozwiązanie jest podane __wyłącznie__ jako kombinacja funkcji jądra $k(x,x_k)$
* w tej postaci odwracana jest macierz $K+\lambda{}I)$
  * jest $N\times{}N$ 
  * oryginalnie odwracana była macierz $\Phi^T\Phi$ o wymiarach $N\times{}M$
  * zwykle $M<N$
  * jednak zyski przeważają nad wadami

## Jak zdefiniować funkcję jądra
1. poprzez $\phi()$ 
$$k(x,y)=\phi(x)^T\phi(y)=\sum\phi_i(x)\phi_j(y)$$
  * to jednak wymaga zdefiniowania $\phi$ i złożonych obliczeń
2. __wprost__ $$k(x,y)=(x^Ty)^2$$
  * to wymaga pewności, że tak zdefiniowana funkcja jest poprawna
    * konieczna jest pewność, że dla tak zdefiniowanej funkcji __istnieje__ jakaś funkcja $\phi()$ taka, że można ją pokazać w postaci 1.
    * taka macierz musi być dodatnio półokreślona
    - [x] opisać czym jest dodatnia półokreśloność

### Pośrednie mapowanie do przestrzeni cech
* budująć model liniowy w przestrzeni $M$-wymiarowej cech szukamy __hipotezy__
$$\boxed{h(x)=\sum_{k=1}^M\theta_k\phi(x_k)+b\;}$$
* $\phi()$ jest jakimś odwzorowaniem z przestrzeni wejściowej do __przestrzeni cech__
* w postaci __dualnej__ hipoteza jest przedstawiona w postaci __relacji między przykładami uczącymi__ a wektorem $x$ dla którego obliczamy wartość
$$\boxed{h(x)=\sum_{n=1}^N\alpha_ny_n<\phi(x_n)\cdot\phi(x)>+b\;}$$
  * potrzebujemy metody __bezpośredniego__ obliczanie $<\phi(x_n)\cdot\phi(x)>$ __bez__ potrzeby obliczania przestrzeni cech
  > Funkcja __jądra__ $k$ jest funkcją dla $x, y\in X$ $$k(x,y)=<\phi(x_n)\cdot\phi(x)>,$$ gdzie $\phi$ jest odwzorowaniem z $X$ do przestrzeni cech $F$
  * dzięki funkcji jądra wymiar przestrzeni cech __nie będzie__  wpływał na złożoność obliczeniową
    * wymiar przestrzeni cech może być nawet nieskończony
    * nie ma potrzeby znajomości przestrzeni cech byle by istaniała
    * złożoność nie musi być zależna od wymiarowości przestrzeni cech
    * dane są mapowane __nie wprost__ do przestrzeni cech
    * wszystkie informacje o przykładach są przechowywane w macierzy Grama $K$
    * hipoteza będzie mieć teraz postać
    $$\boxed{h(x)=\sum_{n=1}^N\alpha_ny_nk(x_n,x)+b\;}$$

#### Funkcje jądra jako pośrednie mapowanie do przestrzeni cech
---
* iloczyn skalarny $<x\cdot y>$ jest przykładem jądra - funkcja jadra jest jego uogólnieniem
* modele będą zwykle potrzebowały wprowadzenia __nieliniowości__, można więc zaproponować jądro $<x\cdot y>^2$
$$\begin{align}
<x\cdot y>^2&=(x_1y_1+x_2y_2)^2\\
&=x_1^2y_1^2+2x_1y_1x_2y_2+x_2^2y_2^2\\
&=(x_1^2,\sqrt{2}x_1x_2,x_2^2)(y_1^2,\sqrt{2}y_1y_2,y_2^2)
\end{align}$$
  * zawiera wszystkie $\displaystyle \binom{M+p-1}{p}$ jednomianów stopnia $p$ dla $k(x,y)=<x\cdot y>^p$
* zmodyfikowane jądro
$$\begin{align}
<x\cdot y+c>^2&=(x_1y_1+x_2y_2+c)^2\\
&=x_1^2y_1^2+2x_1y_1x_2y_2+2cx_1y_1+x_2^2y_2^2+2cx_2y_2+c^2\\
&=(x_1^2,\sqrt{2}x_1x_2,\sqrt{2c}x_1,x_2^2,\sqrt{2c}x_2,c)(y_1^2,\sqrt{2}y_1y_2,\sqrt{2c}y_1,y_2^2,\sqrt{2c}y_2,c)
\end{align}$$
  * podobne wyprowadzenie można znaleźć gdy wykładnik $p>2$
    * każdy będzie się składał z $\displaystyle \binom{M+p}{p}$ składników  będących jednomianami rzędu do $p$ włącznie

In [25]:
figwidth = 230
figheight = 230
x = np.linspace(0, 1, 100)
aa1 = figure(width=figwidth, height=figheight, title=None)
y = x**2
aa1.line(x, y, line_width=2)

aa2 = figure(width=figwidth, height=figheight, title=None)
x = np.linspace(0, 1, 100)
colors = ['blue', 'red', 'orange', 'navy', 'green']
for k, x2 in enumerate([0.1, 0.3, 0.5, 0.7, 0.9]):
    y = np.sqrt(2) * x * x2
    aa2.line(x, y, line_width=2, color=colors[k])
    aa2.x(x2, 0, color=colors[k])

aa3 = figure(width=figwidth, height=figheight, title=None)
x = np.linspace(0, 1, 100)
y = x ** 2
aa3.line(x, y, line_width=2)

fig = gridplot([[aa1, aa2, aa3]])
show(fig)

### Konieczne warunki na funkcję jądra
* __symetryczność__
$$k(x,y)=<\phi(x)\cdot\phi(y)>=<\phi(y)\cdot\phi(x)>=k(y,x)$$
* __nierówność Cauchego-Schwartza__
$$\begin{align}
k(x,y)^2&=<\phi(x)\cdot\phi(y)>^2\leq\|\phi(x)\|^2\|\phi(y)\|^2\\
&=<\phi(x)\cdot\phi(x)><\phi(y)\cdot\phi(y)>=k(x,x)k(y,y)
\end{align}$$
---
* te warunki __nie są wystarczające__ dla istnienia przestrzeni cech indukowanej przez kernel $k()$
  * niech $K=(k(x_i,x_j))_{i,j=1}^N$
  * macierz $K$ jest symetryczna, ponieważ funkcja $k()$ jest symetryczna
  * jeśli K jest symetryczna, to istnieje 
  $$K=V\Lambda V^T$$
  gdzie 
    * $V$ jest ortogonalną macierzą wektorów własnych
    * $\Lambda$ jest diagonalną macierzą wartości własnych
  * zdefiniujmy $\phi$
  $$\phi(x_k)=\left[\sqrt{\lambda_1}v_{1k},\dots,\sqrt{\lambda_i}v_{ik},\dots,\sqrt{\lambda_M}v_{Mk}\right]$$
  * wtedy
  $$\begin{align}<\phi(x_i)\phi(x_j)>&=\sum_k\sqrt{\lambda_k}v_{ki}\sqrt{\lambda_k}v_{kj}\\
  &=\left(V\Lambda V^T\right)_{ij}=k(x_i,x_j)
  \end{align}$$
  * tak więc macierz Grama $K$ musi mieć __nieujemne__ wartości własne, bo w przeciwnym wypadku dla 
  $$x=\sum_iv_{ki}\phi{x_i}=\sqrt{\Lambda}V^Tv_k$$ 
  dla $\lambda_k<0$ zachodzi
  $$\begin{align}
  \|x\|^2=<x\cdot x>&=v_k^TV\sqrt{\Lambda}\sqrt{\Lambda}V^Tv_k\\
  &=v^T_kKv_k=\lambda_k<0
  \end{align}$$
  co byłoby sprzecznością
 

### A kernele z innych kerneli?
Jeśli $k_1()$ i $k_2()$ są poprawnymi kernelami nad $X$, a $k_3()$ jest poprawnym kernelem nad $\mathbb{R}\times\mathbb{R}$, to następujące funkcje też są poprawnymi kernelami
$$\begin{align}
1.&\hspace{2ex}k(x,y)=k_1(x, y)+k_2(x, y)\\
2.&\hspace{2ex}k(x,y)=c k_1(x,y), \hskip{2ex}c>0\\
3.&\hspace{2ex}k(x,y)=k_1(x,y)k_2(x,y)\\
4.&\hspace{2ex}k(x,y)=q(k_1(x,y))\\
5.&\hspace{2ex}k(x,y)=f(x)f(y), \hskip{3ex}f:X\rightarrow\mathbb{R}\\
6.&\hspace{2ex}k(x,y)=k_3(\phi(x),\phi(y))\\
7.&\hspace{2ex}k(x,y)=x^TAy
\end{align}$$
gdzie 
  * $p()$ jest wielomianem z dodatnimi współczynnikami
  * $A$ jest dodatnio półokreśloną macierzą.

* $k(x,y)=\exp(k_1(x,y))$ jest kernelem
  * funkcja wykładnicza może być dowolnie dokładnie przybliżona wielomianami o dodatnich wykładnikach
* 

### Przykładowe jądra
* Gausowskie
$$k(x,y)=\exp\left(-\frac{\|x-y\|^2}{2\sigma^2}\right)$$
dla $\sigma>0$
  * wymiar generowanej przestrzeni cech jest nieskończony
* wielomianowe jednorodne
$$k(x,y)=<x\cdot y>^d$$
* wielomianowe niejednorodne
$$k(x,y)=<x\cdot y + c>^d$$
  * wymiar przestrzeni cech dla jąder wielomianowych jest skończony
* Laplace'a
$$k(x,y)=\exp(t\|x-y\|)$$
dla $t>0$
  * wymiar przestrzeni cech nieskończony
* spline-wskie
$$k(x,y)=M_{2p+1}(\|x-y\|^2)$$
gdzie $B$ jest funkcją spline-owską zdefiniowaną przez konwolucje
* samplujące
$$sinc(x)=\frac{\sin \pi x}{\pi x}$$
szczególnie przydatne dla przetwarzania sygnałów

### Kernele z cech
* innym sposobem jest utworzenie kerneli __z cech__
  * trzeba najpierw wyliczyć cechy, 
  * potem utworzyć iloczyn skalarny
  * __nie ma__ konieczności kontroli półokreśloności macierzy, ponieważ wynika ona bezpośrednio z definicji iloczynu
  
#### Kernel dla łańcuchów
* dany jest alfabet $\Sigma$
  * $s$ i $t$ są łańcuchami w $\Sigma^n$
  * podsekwencja $u$ łańcucha $s$ jest określona przez __multi-indeks__ $i=(i_1,i_2,\dots,i_{|u|})$ 
    * litery w $u$ występują w kolejności rosnącej w $s$, ale mogą wystąpić pomiędzy nimi przerwy
* określamy cechę $\phi_u(s)$ jako
  $$\phi_u(s)=\sum_{i:u=s[i]}\lambda^{l(i)}$$
  gdzie $\lambda\leq1$, $l(i)$ jest długością podsekwencji $u$ i jest sumą po wszystkich multi-indeksach odpowiadających $u$ w łańcuchu $s$
  * $l(i)$ jest liczbą znaków łańcucha $s$ na których jest rozłożona podsekwencja
    * $l(i)$ __nie jest__ równoważne $|u|$
  * cechy mierzą liczbę podsekwencji w łańcuchu biorąc pod uwagę rozproszenie (przerwy)
* funkcja jądra
$$\begin{align}
k_n(s,t)&=\sum_{u\in\Sigma^n}<\phi_u(s)\cdot \phi_u(t)>\\
&=\sum_{u\in\Sigma^n}\sum_{i:u=s[i]}\lambda^{l(i)}\sum_{j:u=t[j]}\lambda^{l(j)}\\
&=\sum_{u\in\Sigma^n}\sum_{i:u=s[i]}\sum_{j:u=t[j]}\lambda^{l(i)+\lambda(j)}
\end{align}$$
sumuje się po wszystkich wspólnych podsekwencjach 
  * brana jest pod uwagę ich częstotliwość i rozproszenie
  
  
  
* taka definicja jest bardzo złozona obliczeniowo
  * możliwa jest jednak definicja rekurencyjna ze względu na indeks $n$ kernela
    * jej złożoność jest proporcjonalna do $n*|s|*|t|$